# TF-IDFの計算

In [1]:
import warnings
warnings.filterwarnings('ignore')

from utils import datasource, helper

query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX imas: <https://sparql.crssnky.xyz/imasrdf/URIs/imas-schema.ttl#>
SELECT ?label
WHERE{
  ?s rdf:type imas:Idol;
  rdfs:label ?label;
  imas:Title ?title.
  FILTER (regex(?title, "765AS") || regex(?title, "MillionStars"))
}
"""

# ニコニコ大百科からのデータ取得
df_idols = datasource.imasparql(query)
df_niconico = datasource.niconico_dic(df_idols['label'].values)

import pandas as pd
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame({
    'label': df_idols['label'].values,
    'text': df_niconico['text']
})
df.head()

,label,text
0,萩原雪歩,ニコニコ大百科\n \n萩原雪歩（はぎわら ゆきほ）は、バンダイナムコエンターテインメントのゲーム『アイドルマスター』（The Idolm@ster）シリーズ...
1,永吉昴,ニコニコ大百科\n \n「永吉昴」（ながよし すばる）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場人物...
2,木下ひなた,ニコニコ大百科\n \n「木下ひなた」（きのした ひなた）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場...
3,伊吹翼,ニコニコ大百科\n \n「伊吹翼」（いぶき つばさ）とは、バンダイナムコエンターテインメントのゲーム『アイドルマスター ミリオンライブ！』シリーズの登場人物で...
4,白石紬,ニコニコ大百科\n \n「白石紬」（しらいし つむぎ）とは、バンダイナムコエンターテインメントの『アイドルマスター ミリオンライブ！』シリーズの登場アイドルで...


In [2]:
# 形態素解析
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()
tokenize = helper.tokenize_func(tokenizer)
df['tokenized'] = df['text'].apply(tokenize)

# Word2Vecモデルの準備
w2v_model = helper.w2v_pretrained_model()
word_list = w2v_model.wv.index2word

# TF-IDFの計算
df['tfidf'] = helper.tfidf_wordlist(df['tokenized'], word_list)
df['tfidf_top'] = helper.wordlist_top_n(df['tfidf'], word_list)

# フォーマットと並べ替え(アイドル順)
def format_tfidf(x):
    return ','.join(["{}({:.2f})".format(k[0], k[1]) for k in x])
df['tfidfstr'] = df['tfidf_top'].apply(format_tfidf)
df_id = datasource.idol_id()
df_orderd = df_id.merge(df, left_on="label", right_on="label").sort_values("id")

# 表示
df_orderd[['label', 'tfidfstr']]

,label,tfidfstr
0,天海春香,"閣下(0.44),春香(0.43),公式(0.11),キャラクター(0.11),ある(0.11),キャラ(0.10),ドジ(0.09),作品(0.09),はる(0.07),受賞(0.07),ド..."
1,如月千早,"千早(0.46),如月(0.20),なる(0.17),ネタ(0.16),ボーカル(0.12),公式(0.12),バースト(0.11),作品(0.10),ある(0.10),テンション(0.10)..."
2,星井美希,"シナリオ(0.18),プロ(0.13),移籍(0.13),分岐(0.10),ムーン(0.10),ある(0.09),キャラクター(0.09),千早(0.09),インタビュー(0.07),設定(0..."
3,萩原雪歩,"ある(0.14),ネタ(0.14),ゆき(0.12),すぐ(0.10),指摘(0.10),ハート(0.10),ゆく(0.10),なる(0.09),キャラ(0.08),しかし(0.08),コメン..."
4,高槻やよい,"うつ(0.14),なる(0.13),ある(0.12),部分(0.11),長女(0.11),節約(0.11),漢字(0.09),歌詞(0.09),ハイ(0.09),デュオ(0.08),あずさ(0..."
5,菊地真,"まく(0.23),ネタ(0.13),同性(0.12),相性(0.12),男性(0.11),ある(0.11),ボク(0.11),コラボ(0.11),運動(0.11),テンション(0.10),認識..."
6,水瀬伊織,"ジュース(0.22),みかん(0.14),その(0.12),ある(0.11),能力(0.10),千早(0.09),趣味(0.09),社長(0.09),なる(0.08),ツッコミ(0.08),プ..."
7,四条貴音,"四条(0.23),ある(0.17),プロ(0.13),設定(0.12),京都(0.12),王女(0.12),銀色(0.12),ライバル(0.11),ステータス(0.11),表現(0.10),キ..."
8,秋月律子,"スレ(0.16),眼鏡(0.15),最後(0.12),事務(0.12),プロ(0.11),その(0.10),ファン(0.10),ブラウス(0.09),一般(0.09),自作(0.09),ローソ..."
9,三浦あずさ,"あずさ(0.27),三浦(0.18),ある(0.12),タイム(0.10),成人(0.09),スカート(0.09),カジュアル(0.09),記念(0.09),年上(0.09),チェック(0.0..."
